In [3]:
import re
import logging
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import pickle
from collections import defaultdict
from utils.utils import prepare, convert_tokens_to_words
from data.configs.demo import config
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel
from utils.Manager import Manager

from models.Embeddings.BERT import BERT_Embedding
from models.Encoders.CNN import CNN_Encoder,CNN_User_Encoder
from models.Encoders.RNN import RNN_Encoder,RNN_User_Encoder
from models.Encoders.MHA import MHA_Encoder, MHA_User_Encoder
from models.Modules.DRM import Matching_Reducer, Slicing_Reducer
from models.Rankers.BERT import BERT_Onepass_Ranker, BERT_Original_Ranker
from models.Rankers.CNN import CNN_Ranker

from models.Encoders.BERT import BERT_Encoder
from models.Encoders.Pooling import *

from models.ESM import ESM
from models.TTMS import TTMS
 
from models.Modules.Attention import MultiheadAttention

# m = AutoModel.from_pretrained('microsoft/deberta-base',cached_path='../../../Data/bert_cache')
t = AutoTokenizer.from_pretrained('microsoft/deberta-base',cached_path='../../../Data/bert_cache')

[2021-09-03 17:14:08,183] INFO (filelock) Lock 139946821365232 acquired on /home/zhengliu/.cache/huggingface/transformers/c2bc27a1c7529c177696ff76b1e74cba8667be14e202359f20f9114e407f43e2.a39abb1c6179fb264c2db685f9a056b7cb8d4bc48d729888d292a2280debf8e2.lock


Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

[2021-09-03 17:14:09,128] INFO (filelock) Lock 139946821365232 released on /home/zhengliu/.cache/huggingface/transformers/c2bc27a1c7529c177696ff76b1e74cba8667be14e202359f20f9114e407f43e2.a39abb1c6179fb264c2db685f9a056b7cb8d4bc48d729888d292a2280debf8e2.lock
[2021-09-03 17:14:11,887] INFO (filelock) Lock 139946733152864 acquired on /home/zhengliu/.cache/huggingface/transformers/ce0ac094af27cf80bbf403595a6d47f1fc632981bf1d4c5bf69968568cbea410.e8ad27cc324bb0dc448d4d95f63e48f72688fb318a4c4c3f623485621b0b515c.lock


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

[2021-09-03 17:14:14,035] INFO (filelock) Lock 139946733152864 released on /home/zhengliu/.cache/huggingface/transformers/ce0ac094af27cf80bbf403595a6d47f1fc632981bf1d4c5bf69968568cbea410.e8ad27cc324bb0dc448d4d95f63e48f72688fb318a4c4c3f623485621b0b515c.lock
[2021-09-03 17:14:14,964] INFO (filelock) Lock 139946733089168 acquired on /home/zhengliu/.cache/huggingface/transformers/05056f257c8d2b63ad16fd26f847c9ab9ee34e33cdfad926e132be824b237869.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

[2021-09-03 17:14:16,958] INFO (filelock) Lock 139946733089168 released on /home/zhengliu/.cache/huggingface/transformers/05056f257c8d2b63ad16fd26f847c9ab9ee34e33cdfad926e132be824b237869.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


In [5]:
t.convert_tokens_to_ids('[SEP]')

2

In [3]:
config.reducer = 'entity'
manager = Manager(config)
loaders = prepare(manager)
x1 = list(loaders[0])[0]
x2 = list(loaders[1])[0]

[2021-09-02 16:37:37,709] INFO (utils.utils) Hyper Parameters are 
scale:demo
batch_size:5
k:5
threshold:-inf
signal_length:100
his_size:50
impr_size:10
lr:0.0001
bert_lr:3e-05
hidden_dim:384
world_size:0
step:0
ascend_history:False
no_dedup:False
diversify:False
granularity:avg
[2021-09-02 16:37:37,710] INFO (utils.utils) preparing dataset...
[2021-09-02 16:37:37,713] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDdemo_train/10/behaviors..pkl
[2021-09-02 16:37:37,725] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDdemo_train/entity.pkl
[2021-09-02 16:37:38,438] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDdemo_dev/10/behaviors..pkl
[2021-09-02 16:37:38,440] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDdemo_dev/entity.pkl


In [6]:
class TTMS(nn.Module):
    def __init__(self, config, embedding, encoderN, encoderU, reducer):
        super().__init__()

        self.scale = config.scale
        self.cdd_size = config.cdd_size
        self.batch_size = config.batch_size
        self.his_size = config.his_size
        self.signal_length = config.signal_length
        self.device = config.device

        self.embedding = embedding
        self.encoderN = encoderN
        self.encoderU = encoderU

        self.reducer = reducer
        self.bert = BERT_Encoder(config)

        self.granularity = config.granularity
        if self.granularity != 'token':
            self.register_buffer('cdd_dest', torch.zeros((self.batch_size, config.impr_size, config.signal_length * config.signal_length)), persistent=False)
            if config.reducer in ["bm25", "entity", "first"]:
                self.register_buffer('his_dest', torch.zeros((self.batch_size, self.his_size, (config.k + 1) * (config.k + 1))), persistent=False)
            else:
                self.register_buffer('his_dest', torch.zeros((self.batch_size, self.his_size, config.signal_length * config.signal_length)), persistent=False)

        self.userProject = nn.Sequential(
            nn.Linear(self.bert.hidden_dim, self.bert.hidden_dim),
            nn.Tanh()
        )

        self.register_buffer('extra_cls_mask', torch.ones(1,1), persistent=False)

        self.name = '__'.join(['ttms', self.encoderN.name, self.encoderU.name, config.reducer, self.granularity])
        config.name = self.name

    def clickPredictor(self, cdd_news_repr, user_repr):
        """ calculate batch of click probabolity

        Args:
            cdd_news_repr: news-level representation, [batch_size, cdd_size, hidden_dim]
            user_repr: user representation, [batch_size, 1, hidden_dim]

        Returns:
            score of each candidate news, [batch_size, cdd_size]
        """
        # print(user_repr.mean(), cdd_news_repr.mean(), user_repr.max(), cdd_news_repr.max(), user_repr.sum(), cdd_news_repr.sum())
        score = cdd_news_repr.matmul(user_repr.transpose(-2,-1)).squeeze(-1)
        return score

    def _forward(self,x):
        if self.granularity != 'token':
            batch_size = x['cdd_subword_index'].size(0)
            cdd_size = x['cdd_subword_index'].size(1)

            if self.training:
                if batch_size != self.batch_size:
                    cdd_dest = self.cdd_dest[:batch_size, :cdd_size]
                    his_dest = self.his_dest[:batch_size]
                else:
                    cdd_dest = self.cdd_dest[:, :cdd_size]
                    his_dest = self.his_dest

            # batch_size always equals 1 when evaluating
            else:
                cdd_dest = self.cdd_dest[[0], :cdd_size]
                his_dest = self.his_dest[[0]]

            cdd_subword_index = x['cdd_subword_index'].to(self.device)
            his_subword_index = x['his_subword_index'].to(self.device)
            his_signal_length = his_subword_index.size(-2)
            cdd_subword_index = cdd_subword_index[:, :, :, 0] * self.signal_length + cdd_subword_index[:, :, :, 1]
            his_subword_index = his_subword_index[:, :, :, 0] * his_signal_length + his_subword_index[:, :, :, 1]

            if self.training:
                cdd_subword_prefix = cdd_dest.scatter(dim=-1, index=cdd_subword_index, value=1) * x["cdd_mask"].to(self.device)
            else:
                cdd_subword_prefix = cdd_dest.scatter(dim=-1, index=cdd_subword_index, value=1)
            cdd_subword_prefix = cdd_subword_prefix.view(batch_size, cdd_size, self.signal_length, self.signal_length)

            his_subword_prefix = his_dest.scatter(dim=-1, index=his_subword_index, value=1) * x["his_mask"].to(self.device)
            his_subword_prefix = his_subword_prefix.view(batch_size, self.his_size, his_signal_length, his_signal_length)

            if self.granularity == 'avg':
                # average subword embeddings as the word embedding
                cdd_subword_prefix = F.normalize(cdd_subword_prefix, p=1, dim=-1)
                his_subword_prefix = F.normalize(his_subword_prefix, p=1, dim=-1)

            cdd_attn_mask = cdd_subword_prefix.matmul(x['cdd_attn_mask'].to(self.device).float().unsqueeze(-1)).squeeze(-1)
            his_attn_mask = his_subword_prefix.matmul(x["his_attn_mask"].to(self.device).float().unsqueeze(-1)).squeeze(-1)
            his_refined_mask = None
            if 'his_refined_mask' in x:
                his_refined_mask = his_subword_prefix.matmul(x["his_refined_mask"].to(self.device).float().unsqueeze(-1)).squeeze(-1)

        else:
            cdd_subword_prefix = None
            his_subword_prefix = None
            cdd_attn_mask = x['cdd_attn_mask'].to(self.device)
            his_attn_mask = x["his_attn_mask"].to(self.device)
            his_refined_mask = None
            if 'his_refined_mask' in x:
                his_refined_mask = x["his_refined_mask"].to(self.device)


        cdd_news = x["cdd_encoded_index"].long().to(self.device)
        _, cdd_news_repr = self.bert(
            self.embedding(cdd_news, cdd_subword_prefix), cdd_attn_mask
        )

        his_news = x["his_encoded_index"].long().to(self.device)
        his_news_embedding = self.embedding(his_news, his_subword_prefix)
        his_news_encoded_embedding, his_news_repr = self.encoderN(
            his_news_embedding
        )
        # no need to calculate this if ps_terms are fixed in advance
        if self.reducer.name == 'matching':
            user_repr = self.encoderU(his_news_repr)
        else:
            user_repr = None

        ps_terms, ps_term_mask, kid = self.reducer(his_news_encoded_embedding, his_news_embedding, user_repr, his_news_repr, his_attn_mask, his_refined_mask)

        # append CLS to the entire browsing history, directly deriving user repr
        batch_size = ps_terms.size(0)
        ps_terms = torch.cat([his_news_embedding[:, 0, 0].unsqueeze(1), ps_terms], dim=-2)
        ps_term_mask = torch.cat([self.extra_cls_mask.expand(batch_size, 1), ps_term_mask], dim=-1)
        _, user_cls = self.bert(ps_terms.unsqueeze(1), ps_term_mask.unsqueeze(1))
        user_repr = self.userProject(user_cls)

        return self.clickPredictor(cdd_news_repr, user_repr), kid

    def forward(self,x):
        """
        Decoupled function, score is unormalized click score
        """
        score, kid = self._forward(x)

        if self.training:
            prob = nn.functional.log_softmax(score, dim=1)
        else:
            prob = torch.sigmoid(score)

        return prob, kid

In [8]:
embedding = BERT_Embedding(manager)

encoderN = CNN_Encoder(manager)
# encoderN = RNN_Encoder(manager)
# encoderN = MHA_Encoder(manager)

# encoderU = CNN_User_Encoder(manager)
encoderU = RNN_User_Encoder(manager)
# encoderU = MHA_User_Encoder(manager)

# reducer = Matching_Reducer(manager)
reducer = Slicing_Reducer(manager)

# ranker = CNN_Ranker(manager)
# ranker = BERT_Onepass_Ranker(manager)
# ranker = BERT_Original_Ranker(manager)

model = TTMS(manager, embedding, encoderN, encoderU, reducer).to(manager.device)

In [9]:
t = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
a,b,c = model(x1)

In [17]:
x3 = list(loaders[0])[-1]

In [19]:
model.train()
model(x3)

In [16]:
x3['his_subword_index'].max()

tensor(5)

In [40]:
toks = t.convert_ids_to_tokens(x1['cdd_encoded_index'][0,1])
word = convert_tokens_to_words(toks)

In [52]:
x1['his_attn_mask'][0,1], x1['his_attn_mask'][0,1].sum()

(tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 tensor(58))

In [54]:
c[2][0,1], c[2][0,1].sum()

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor(53.))

In [66]:
x1['his_refined_mask'][0,1], torch.arange(100)[c[3][0,1] == 0.75], c[3][0,1]

(tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
         0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 tensor([52]),
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         0.0000, 1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 1.0000,
         1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 1.0000,
         1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.7500, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000

In [60]:
toks = [i for i in t.convert_ids_to_tokens(x1['his_encoded_index'][0,1]) if i != '[PAD]']
word = convert_tokens_to_words(toks)

len(word), len(toks)

(53, 58)